<a href="https://colab.research.google.com/github/Sirabhop/Preclinical-AD-EEG-classification/blob/master/Machine_Learning_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import os
import pandas as pd

In [0]:
#Set working directory and import cookbook
os.chdir("/content/drive/My Drive/Colab Notebooks/")

#Download eeg feature extraction function
%run EEG_Feature_Extraction.ipynb

# **Import Data**

In [0]:
drive.mount('/content/drive')

In [0]:
os.chdir("/content/drive/My Drive/EEG Data")

In [0]:
#Compute EEG feature
subjectID_fif = ['SS1', 'SS2', 'SS3']
df = getDataFrame_fif(subjectID_fif)

subjectID_bdf = ['SS4', 'SS5', 'SS6', 'SS7', 'SS8']
df2 = getDataFrame_bdf(subjectID_bdf)

df = df.append(df2, ignore_index = True)

#Export EEG feature
df.to_csv('EEG_features.csv')

In [0]:
#Import FCSRT & MoCA & Demographic data
FCSRT = pd.read_csv("FCSRT.csv")
FCSRT.drop('id', axis = 1, inplace = True)
MoCA = pd.read_csv("MoCA.csv", usecols = ['SUM'])
EEG = pd.read_csv("EEG_features.csv")
Group = pd.DataFrame(columns = ['Group'])

df_original = pd.concat([EEG,FCSRT,MoCA,Group], axis = 1)
df_original.drop(['Unnamed: 0'], inplace = True, axis = 1)
df_original.rename(columns = {'SUM':'MoCA'}, inplace = True)
df_original.drop('id', axis = 1, inplace=True)

# **Data Preprocessing**

#### **1) Scaling**

In [0]:
from sklearn import preprocessing

In [77]:
#Min-Max Scaling

scaler = preprocessing.MinMaxScaler()
df_original[df_original.columns] = scaler.fit_transform(df_original[df_original.columns])

df_original

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:375: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:376: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


,PSD_Delta,PSD_Theta,PSD_Alpha,PSD_Beta,PSD_Gamma,MSF,SE,AC,wSMI_Alpha,wSMI_Theta,IR-FreeR,IR-CuedR,DL-FreeR,DL-CuedR,MoCA,Group
0,NaN,0.003081,0.003081,0.011514,0.078478,0.032240,1.000000,0.000000,0.000000,0.000000,0.0,1.00,0.2,0.333333,0.9375,NaN
1,NaN,0.000267,0.000267,0.002817,0.000000,0.227158,0.654121,0.302423,0.349803,0.328528,0.2,0.25,0.6,0.333333,0.0625,NaN
2,NaN,0.000495,0.000495,0.024244,0.346150,0.569739,0.724031,0.972203,0.531520,0.528180,1.0,0.00,1.0,0.000000,0.1875,NaN
3,NaN,1.000000,1.000000,1.000000,1.000000,0.381145,0.654121,0.253809,0.682812,0.678925,1.0,0.00,1.0,0.333333,1.0000,NaN
4,NaN,0.000302,0.000302,0.003871,0.006691,0.294278,0.000000,0.680209,0.795899,0.797631,0.8,0.00,0.2,1.000000,0.3750,NaN
5,NaN,0.000000,0.000000,0.000000,0.004384,0.000000,0.000000,0.400103,0.884533,0.892868,0.0,0.00,0.0,0.000000,0.5625,NaN
6,NaN,0.001848,0.001848,0.016655,0.179131,1.000000,0.801038,0.483071,0.946392,0.950359,0.4,0.75,0.6,0.666667,0.8750,NaN
7,NaN,0.000046,0.000046,0.004230,0.045855,0.225229,0.000000,1.000000,1.000000,1.000000,0.8,0.25,0.6,1.000000,0.0000,NaN


#### **2) Generate Data**

The section below, I've random the value of each feature in order to demonstrate ML model because the situation of *COVID-19* that I cannot continue run the experiment as expected.

In [0]:
from random import random

In [0]:
df_random = pd.DataFrame(columns = df_original.columns)
for x in df_original.columns:
  df_random[x] = np.random.rand(52)

In [113]:
df = df_original.append(df_random, ignore_index = True)
df.tail()

,PSD_Delta,PSD_Theta,PSD_Alpha,PSD_Beta,PSD_Gamma,MSF,SE,AC,wSMI_Alpha,wSMI_Theta,IR-FreeR,IR-CuedR,DL-FreeR,DL-CuedR,MoCA,Group
55,0.081048,0.157853,0.368764,0.510454,0.694992,0.438233,0.569130,0.717724,0.299608,0.779908,0.067550,0.826314,0.075395,0.016342,0.554737,0.686182
56,0.639774,0.446899,0.932513,0.769096,0.028778,0.403011,0.903073,0.821025,0.125291,0.175834,0.306016,0.063432,0.215287,0.165735,0.897453,0.950990
57,0.628887,0.118208,0.282147,0.326083,0.275932,0.778104,0.815768,0.630028,0.022750,0.711233,0.151551,0.410624,0.330279,0.917817,0.657837,0.675762
58,0.857436,0.159720,0.287330,0.084511,0.326493,0.780019,0.426419,0.964238,0.894543,0.917505,0.691122,0.705839,0.624885,0.933273,0.875307,0.195411
59,0.814756,0.712513,0.406077,0.747590,0.182451,0.701558,0.077096,0.030490,0.776659,0.512631,0.288686,0.402172,0.330336,0.484330,0.659829,0.192102
